In [1]:
from kolmov import crossval_table, crossval_fit_table, get_color_fader
from Gaugi import mkdir_p
from kepler import load as kload
from sklearn.model_selection import StratifiedKFold, KFold
from Gaugi import load as gload

import json
from itertools import product
from tqdm import tqdm
import tensorflow as tf

import saphyra
import numpy as np
import pandas as pd
import collections
import os
import matplotlib
import matplotlib.pyplot as plt
from pprint import pprint
from copy import deepcopy, copy
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00


2022-01-10 01:54:33.533382: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


Using all sub packages with ROOT dependence

Applying ATLAS style settings...

Applying ATLAS style settings...


2022-01-10 01:54:34.726681: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-01-10 01:54:34.771145: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-10 01:54:34.771503: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.515GHz coreCount: 14 deviceMemorySize: 3.82GiB deviceMemoryBandwidth: 178.84GiB/s
2022-01-10 01:54:34.771538: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-01-10 01:54:34.775877: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-01-10 01:54:34.776760: I tensorflow/stream_executor/platform/d

In [2]:

def create_op_dict(op):
    d = {
              op+'_pd_ref'    : "reference/"+op+"_cutbased/pd_ref#0",
              op+'_fa_ref'    : "reference/"+op+"_cutbased/fa_ref#0",
              op+'_sp_ref'    : "reference/"+op+"_cutbased/sp_ref",
              op+'_pd_val'    : "reference/"+op+"_cutbased/pd_val#0",
              op+'_fa_val'    : "reference/"+op+"_cutbased/fa_val#0",
              op+'_sp_val'    : "reference/"+op+"_cutbased/sp_val",
              op+'_pd_op'     : "reference/"+op+"_cutbased/pd_op#0",
              op+'_fa_op'     : "reference/"+op+"_cutbased/fa_op#0",
              op+'_sp_op'     : "reference/"+op+"_cutbased/sp_op",

              # Counts
              op+'_pd_ref_passed'    : "reference/"+op+"_cutbased/pd_ref#1",
              op+'_fa_ref_passed'    : "reference/"+op+"_cutbased/fa_ref#1",
              op+'_pd_ref_total'     : "reference/"+op+"_cutbased/pd_ref#2",
              op+'_fa_ref_total'     : "reference/"+op+"_cutbased/fa_ref#2",
              op+'_pd_val_passed'    : "reference/"+op+"_cutbased/pd_val#1",
              op+'_fa_val_passed'    : "reference/"+op+"_cutbased/fa_val#1",
              op+'_pd_val_total'     : "reference/"+op+"_cutbased/pd_val#2",
              op+'_fa_val_total'     : "reference/"+op+"_cutbased/fa_val#2",
              op+'_pd_op_passed'     : "reference/"+op+"_cutbased/pd_op#1",
              op+'_fa_op_passed'     : "reference/"+op+"_cutbased/fa_op#1",
              op+'_pd_op_total'      : "reference/"+op+"_cutbased/pd_op#2",
              op+'_fa_op_total'      : "reference/"+op+"_cutbased/fa_op#2",
    }
    return d

tuned_info = collections.OrderedDict( {
              # validation
              "max_sp_val"      : 'summary/max_sp_val',
              "max_sp_pd_val"   : 'summary/max_sp_pd_val#0',
              "max_sp_fa_val"   : 'summary/max_sp_fa_val#0',
              # Operation
              "max_sp_op"       : 'summary/max_sp_op',
              "max_sp_pd_op"    : 'summary/max_sp_pd_op#0',
              "max_sp_fa_op"    : 'summary/max_sp_fa_op#0',
              } )

op_names = ['tight', 'medium', 'loose', 'vloose']

tuned_info = collections.OrderedDict({})
for op in op_names:
    tuned_info[op] = create_op_dict(op)

In [3]:
etbins  = [4, 7, 10, 15]
etabins = [0.0, 0.8, 1.37, 1.54, 2.37, 2.50]
min_avgmu = 16
max_avgmu = 100

l_path  = '/home/micael/Documents/NeuralRinger/Jpsiee'

tunes_path    = os.path.join(l_path, 'tunes')
analysis_path = os.path.join(l_path, 'analysis')

output_path   = os.path.join(analysis_path, 'v1/r0/output/fitting')
mkdir_p(output_path)

In [4]:
cv  = crossval_table( tuned_info, etbins = etbins , etabins = etabins )

In [5]:
cv.from_csv(os.path.join(analysis_path, 'v1/r0/table_v1.csv'))

In [6]:
best_inits = cv.filter_inits("max_sp_val")
best_inits = best_inits.loc[(best_inits.train_tag == 'v1.r0.mlp2' )] # 5 neurons for all phase spaces
best_sorts = cv.filter_sorts( best_inits , 'max_sp_op')
best_inits.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,op_name,max_sp_val,...,pd_ref_total,fa_ref_total,pd_val_passed,fa_val_passed,pd_val_total,fa_val_total,pd_op_passed,fa_op_passed,pd_op_total,fa_op_total
1310,v1.r0.mlp2,0,0,0,0,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,loose,0.937434,...,28455,215903,2783,2437,2846,21590,27827,23588,28455,215903
1309,v1.r0.mlp2,0,0,0,0,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,medium,0.937434,...,28455,215903,2799,2708,2846,21590,27987,27206,28455,215903
1308,v1.r0.mlp2,0,0,0,0,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,tight,0.937434,...,28455,215903,2799,2708,2846,21590,27987,27206,28455,215903
1311,v1.r0.mlp2,0,0,0,0,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,vloose,0.937434,...,28455,215903,2815,3897,2846,21590,28149,33945,28455,215903
1138,v1.r0.mlp2,0,0,0,1,1,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,loose,0.940628,...,28455,215903,2783,2490,2846,21590,27827,24306,28455,215903


## 2) Linear correction:

In [ ]:
datapath = os.path.join(l_path, 'data/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM2.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.15bins/' +\
                                                'data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM2.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.15bins_et{ET}_eta{ETA}.npz')
paths = [ [datapath.format(ET=et_bin, ETA=eta_bin) for eta_bin in range(5)] for et_bin in range(3)]

In [7]:
def my_data_generator( path, pidname='el_lhmedium' ):  
    import pandas as pd
    from kepler import load
    df = load(path)
    # NOTE: Offline filter lhvloose -> lhmedium (as the training procedure)
    df = df.loc[ ((df[pidname]==True) & (df.target==1.0)) | ((df.target==0) & (df['el_lhvloose']==0)) ]
    return df


def my_input_generator( df ):
    '''
    This function will generate the input for ringer models.

    Arguments:
    df: the dataframe
    '''
    col_names= ['trig_L2_cl_ring_%d'%i for i in range(100)]
    rings = df[col_names].values.astype(np.float32)
    
    def norm1( data ):
        norms = np.abs( data.sum(axis=1) )
        norms[norms==0] = 1
        return data/norms[:,None]
    
    rings = norm1(rings)
    return [rings]

def my_dec_generator_after( row, df , remove_last=True):
    
    from Gaugi import load as gload
    file_name  = row.file_name.values[0]
    i_init     = row.init.values[0]
    i_sort     = row.sort.values[0]
    model_idx  = row.model_idx.values[0]
    tuned_data = gload(file_name)['tunedData']
    slope = row.slope.values[0]
    offset = row.offset.values[0]
    for ituned in tuned_data:
        if ((ituned['imodel'] == model_idx)\
            and (ituned['sort'] == i_sort) \
            and (ituned['init'] == i_init)):
            sequence, weights = ituned['sequence'], ituned['weights']
    model = tf.keras.models.model_from_json( json.dumps(sequence, separators=(',', ':')) ) #custom_objects={'RpLayer':RpLayer} )
    model.set_weights( weights )
    if remove_last:
        model = tf.keras.models.Model(model.inputs, model.layers[-2].output)
        
    output = model.predict( my_input_generator(df), batch_size=4096 ).flatten()
    avgmu = df.avgmu.values.flatten()
    avgmu[avgmu>max_avgmu] = max_avgmu
    avgmu[avgmu<min_avgmu] = min_avgmu
    dec = np.greater(output, slope*avgmu + offset)
    return dec

def my_dec_generator( row, df ):
    '''
    This function will load a model from dataframe get their decision and return.

    Arguments:
    row: a dataframe row (used by kolmov)
    df:  the dataframe that will be use to get the decision
    '''
    from Gaugi import load as gload
    file_name  = row.file_name.values[0]
    i_init     = row.init.values[0]
    i_sort     = row.sort.values[0]
    model_idx  = row.model_idx.values[0]
    op_name    = row.op_name.values[0]
    tuned_data = gload(file_name)['tunedData']
    for ituned in tuned_data:
        if ((ituned['imodel'] == model_idx)\
            and (ituned['sort'] == i_sort) \
            and (ituned['init'] == i_init)):
            sequence, weights = ituned['sequence'], ituned['weights']
            thr = ituned['history']['reference']['%s' %(op_name)]['threshold_op']
    model = tf.keras.models.model_from_json( json.dumps(sequence, separators=(',', ':')) ) #custom_objects={'RpLayer':RpLayer} )
    model.set_weights( weights )
    
    output = model.predict( my_input_generator(df), batch_size=4096 ).flatten()
    dec = np.greater(output, thr)
    return dec

In [ ]:
# calculate all pd/fa from reference file
ref_path = os.path.join(l_path, 'data/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM2.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.15bins/new_references/' +\
                                'data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM2.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.15bins_et{ET}_eta{ETA}.ref.npz')

ref_paths = [[ ref_path.format(ET=et,ETA=eta) for eta in range(5)] for et in range(3)]
ref_values = [[ {} for eta in range(5)] for et in range(3)]

from saphyra.core import ReferenceReader
for et_bin in range(3):
    for eta_bin in range(5):
        for op_name in op_names:
            refObj = ReferenceReader().load(ref_paths[et_bin][eta_bin])
            _pd = refObj.getSgnPassed(op_name )/refObj.getSgnTotal(op_name)
            _fa = refObj.getBkgPassed(op_name )/refObj.getBkgTotal(op_name)
            ref_values[et_bin][eta_bin][op_name] = {'pd':_pd, 'fa':_fa, 'pd_epsilon':0.0}

In [ ]:
kf = StratifiedKFold(n_splits=10, random_state=512, shuffle=True)

**NOTE**: This step will take something like 20min to be ready

In [ ]:
cv_fit = crossval_fit_table( etbins, etabins, kf, my_data_generator, my_input_generator)

In [ ]:
best_inits_after = cv_fit.fill( best_inits , paths, ref_values, output_path=output_path ,
                                min_avgmu=min_avgmu, max_avgmu=max_avgmu, xbin_size=0.001, ybin_size=1.5, ymin=16.5, ymax=45.5, 
                                xmin_percentage=0.05, xmax_percentage=99.95,)

In [ ]:
cv_fit.to_csv(os.path.join(analysis_path, 'v1/r0/output/fitting/best_inits_v1_with_mu_cor.csv'))


In [8]:
best_inits_after = pd.read_csv(os.path.join(analysis_path, 'v1/r0/output/fitting/best_inits_v1_with_mu_cor.csv'))

### 4) Run test:

In [9]:
datapath = os.path.join(l_path, 'data/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM2.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.15bins/' +\
                        'data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM2.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.15bins_et{ET}_eta{ETA}.npz')
test_paths = [ [datapath.format(ET=et_bin, ETA=eta_bin) for eta_bin in range(5)] for et_bin in range(3)]

### 4.1) Before correction:

In [10]:
test_table = cv.evaluate( best_sorts , test_paths, my_data_generator, my_dec_generator)

Fitting... :  60%|████████████▌        | 9/15 [00:08<00:04,  1.24it/s]

Fitting... :  67%|█████████████▎      | 10/15 [00:09<00:03,  1.50it/s]

Fitting... : 100%|████████████████████| 15/15 [00:14<00:00,  1.02it/s]
2022-01-10 01:54:48.257440: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 76044800 exceeds 10% of free system memory.
2022-01-10 01:54:48.359018: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-01-10 01:54:48.759324: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 76044800 exceeds 10% of free system memory.
2022-01-10 01:54:49.056211: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 76044800 exceeds 10% of free system memory.
2022-01-10 01:54:49.357918: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 76044800 exceeds 10% of free system memory.
2022-01-10 01:54:57.226951: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 78559200 exceeds 10% of free system memory.


In [11]:
display(best_inits.head())
display(best_inits_after.head())

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,op_name,max_sp_val,...,pd_ref_total,fa_ref_total,pd_val_passed,fa_val_passed,pd_val_total,fa_val_total,pd_op_passed,fa_op_passed,pd_op_total,fa_op_total
1310,v1.r0.mlp2,0,0,0,0,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,loose,0.937434,...,28455,215903,2783,2437,2846,21590,27827,23588,28455,215903
1309,v1.r0.mlp2,0,0,0,0,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,medium,0.937434,...,28455,215903,2799,2708,2846,21590,27987,27206,28455,215903
1308,v1.r0.mlp2,0,0,0,0,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,tight,0.937434,...,28455,215903,2799,2708,2846,21590,27987,27206,28455,215903
1311,v1.r0.mlp2,0,0,0,0,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,vloose,0.937434,...,28455,215903,2815,3897,2846,21590,28149,33945,28455,215903
1138,v1.r0.mlp2,0,0,0,1,1,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,loose,0.940628,...,28455,215903,2783,2490,2846,21590,27827,24306,28455,215903


,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,op_name,max_sp_val,...,pd_val_total,fa_val_total,pd_op_passed,fa_op_passed,pd_op_total,fa_op_total,slope,offset,fig_signal,fig_background
0,v1.r0.mlp2,0,0,0,0,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,loose,0.937434,...,2846,21590,27815,24343,28455,215903,-0.009778,-0.131798,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,/home/micael/Documents/NeuralRinger/Jpsiee/ana...
1,v1.r0.mlp2,0,0,0,0,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,medium,0.937434,...,2846,21590,27986,28784,28455,215903,-0.010341,-0.313188,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,/home/micael/Documents/NeuralRinger/Jpsiee/ana...
2,v1.r0.mlp2,0,0,0,0,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,tight,0.937434,...,2846,21590,27986,28784,28455,215903,-0.010341,-0.313188,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,/home/micael/Documents/NeuralRinger/Jpsiee/ana...
3,v1.r0.mlp2,0,0,0,0,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,vloose,0.937434,...,2846,21590,28141,35635,28455,215903,-0.014682,-0.409192,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,/home/micael/Documents/NeuralRinger/Jpsiee/ana...
4,v1.r0.mlp2,0,0,0,1,1,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,loose,0.940628,...,2846,21590,27816,25874,28455,215903,-0.012658,-0.027790,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,/home/micael/Documents/NeuralRinger/Jpsiee/ana...


In [12]:
for op_name in op_names:
    table = best_inits.loc[best_inits.op_name == op_name]
    output = 'table_v1_%s_eval_data18.pdf'%op_name
    cv.dump_beamer_table(table, output, tags=['v1.r0.mlp2'], 
                         test_table = test_table.loc[test_table.op_name==op_name],
                         title='Eval data18_13TeV %s v1 tuning (with tanh)'%op_name,
                         cv_tag_name='Ringer (2017)',
                         tuning_ref_name='Cut-Based (2017)',
                         test_name='Ringer (2018)',
                         test_ref_name='Cut-Based (2018)'
                         )

2022-01-10 01:55:05,019 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file table_v1_tight_eval_data18.pdf latex code...
2022-01-10 01:55:05,762 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file table_v1_medium_eval_data18.pdf latex code...
2022-01-10 01:55:06,675 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file table_v1_loose_eval_data18.pdf latex code...
2022-01-10 01:55:07,713 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file table_v1_vloose_eval_data18.pdf latex code...


### 4.2) After correction:

In [15]:
best_inits_after['delta_ref'] = abs(best_inits_after.pd_ref - best_inits_after.pd_op)
best_sorts_after = cv.filter_sorts(best_inits_after, 'delta_ref', idxmin=True )
best_sorts_after.to_csv(os.path.join(analysis_path, 'v1/r0/output/fitting/best_sorts_v1_with_mu_cor.csv'), index=False)

In [16]:
best_sorts_after.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,op_name,max_sp_val,...,fa_val_total,pd_op_passed,fa_op_passed,pd_op_total,fa_op_total,slope,offset,fig_signal,fig_background,delta_ref
24,v1.r0.mlp2,0,0,0,6,4,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,loose,0.939206,...,21591,27825,25582,28455,215903,-0.010263,-0.108604,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,0.000070
1,v1.r0.mlp2,0,0,0,0,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,medium,0.937434,...,21590,27986,28784,28455,215903,-0.010341,-0.313188,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,0.000035
2,v1.r0.mlp2,0,0,0,0,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,tight,0.937434,...,21590,27986,28784,28455,215903,-0.010341,-0.313188,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,0.000035
35,v1.r0.mlp2,0,0,0,8,3,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,vloose,0.941184,...,21590,28148,34616,28455,215903,-0.006295,-0.692980,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,0.000035
44,v1.r0.mlp2,0,1,0,1,4,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,loose,0.923447,...,17908,8532,29537,8736,179082,-0.012149,-0.099207,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,/home/micael/Documents/NeuralRinger/Jpsiee/ana...,0.000114


In [17]:
test_table_after = cv.evaluate( best_sorts_after , test_paths, my_data_generator, my_dec_generator_after )

Fitting... :  60%|████████████▌        | 9/15 [00:08<00:05,  1.15it/s]

Fitting... :  67%|█████████████▎      | 10/15 [00:09<00:03,  1.30it/s]

Fitting... : 100%|████████████████████| 15/15 [00:15<00:00,  1.05s/it]


In [18]:
test_table.to_csv(os.path.join(analysis_path, 'v1/r0/output/fitting/table_v1_test_with_mu_cor.csv'), index=False)

In [21]:
for op_name in op_names:
    table = best_inits_after.loc[best_inits_after.op_name == op_name]
    output = 'table_v1_%s_with_mu_cor.pdf'%op_name
    cv.dump_beamer_table(table, output, tags=['v1.r0.mlp2'], 
                         test_table = test_table_after.loc[test_table_after.op_name==op_name],
                         title='Eval data18_13TeV %s v1 tuning (with correction)'%op_name,
                         cv_tag_name='Ringer (2017)',
                         tuning_ref_name='Cut-Based (2017)',
                         test_name='Ringer (2018)',
                         test_ref_name='Cut-Based (2018)'
                         )

2022-01-10 01:51:02,711 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file table_v1_tight_with_mu_cor.pdf latex code...
2022-01-10 01:51:03,566 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file table_v1_medium_with_mu_cor.pdf latex code...
2022-01-10 01:51:04,453 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file table_v1_loose_with_mu_cor.pdf latex code...
2022-01-10 01:51:05,422 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file table_v1_vloose_with_mu_cor.pdf latex code...


### 1.2) Get threshold configurations: